#  <font color='orange'>CUNEF</font>
# `Machine Learning`
# `Yelp Dataset`
# `01_DataExtraction_Yelp:`
## Authors:
### Jorge Plaza Yuste
### Álvaro Gómez Pérez

## Libraries

In [1]:
# Basic
import json
import pandas as pd
import numpy as np
from collections import Counter

## Functions

#### Data loading and saving

In [2]:
def load_dict(file, sheets):
    """
    Load the data dicts from excel format.
    The output is a list of pandas dataframes.
    """    
    # Dicts list:
    dicts = []
    
    # Iteration for loading each dict:
    for i in sheets:
        dictionary = pd.read_excel(file, sheet_name = i)
        dicts.append(dictionary)
        
    return dicts

def load_data(files, dtypes):
    """
    Load the data from json format.
    The output is a list of pandas dataframes.
    """    
    # Dataframe list:
    data = []
    
    # Iteration for loading each file:
    for i in range(0,len(files)):
        
        # Empty list to store each chunk
        chunks = []  

        # Store the json chunks
        reader = pd.read_json(files[i],
                              lines=True,
                              chunksize=1000,
                              dtype=dtypes[i],
                              orient='records')

        # For loop to append each chunk into the chunks list
        for chunk in reader:
            
            chunks.append(chunk)

        # Concatenate each chunk into a single dataframe
        df = pd.concat(chunks, ignore_index=True)
        data.append(df)
        
    return data

def save_dfs_to_parquet(dataframes, filenames):
    """
    Save the model to parquet.
    The file path must end with '.pkl'.
    """
    for df, file in zip(dataframes, filenames):
        df.to_parquet(file)

#### Data treatment

In [3]:
def words_counter(df, column, n_words):
    """
    Given a dataframe and a text column, returns a dataframe with the most common words.
    """    
    words = df[pd.notnull(df[column])][column].str.split().tolist()
    
    # Flatten the list of lists into a single list of words
    words = [word for sublist in words for word in sublist]
    
    # Remove the commas from the words
    words = [word.replace(',', '') for word in words]
    
    # Count the frequency of each word
    word_counts = Counter(words)
    
    # Get the 15 most common words
    most_common_words = word_counts.most_common(n_words)
    
    # Return the most common words
    return pd.DataFrame(most_common_words, columns=['word', 'count'])

## Dicts loading and display

In [4]:
# Path to file:
file = '../data/diccionario.xlsx'
# Name of the sheets of the excel:
sheets = ['business', 'checkin', 'tip', 'users', 'reviews']
# Dict loading:
business_dict, checkin_dict, tip_dict, users_dict, reviews_dict = load_dict(file,sheets)

In [5]:
business_dict

,Variable,Descripcion,Tipo
0,business_id,Identificador del negocio,string
1,name,Nombre del negocio,string
2,address,Dirección del negocio,string
3,city,Ciudad donde se encuentra el negocio,string
4,state,Estado donde se encuentra el negocio,string
5,postal code,Codigo postal del negocio,string
6,latitude,Latitud espacial del negocio,float
7,longitude,Longitud espacial del negocio,float
8,stars,Valoración del negocio,float
9,review_count,Numero de criticas del negocio,int


In [6]:
checkin_dict

,Variable,Descripcion,Tipo
0,business_id,Identificador del negocio,string
1,date,Conjunto de fechas en las que se realizo cada ...,string


In [7]:
tip_dict

,Variable,Descripcion,Tipo
0,text,Contenido del consejo o recomendacion,string
1,date,Dia de registro de la recomendación,string
2,compliment_count,Numero de cumplidos que tiene la recomendación,int
3,business_id,Identificador del negocio,string
4,user_id,Identificador del usuario,string


In [8]:
users_dict

,Variable,Descripcion,Tipo
0,user_id,Identificador del usuario,string
1,name,Nombre del usuario,string
2,review_count,Numero de criticas o reseñas realizadas,int
3,yelping_since,Antigüedad del usario en Yelp,string
4,friends,Identificadores de los amigos del usuario,object
5,useful,Numero de votos utiles realizados,int
6,funny,Numero de votos divertidos realizados,int
7,cool,"Numero de votos ""guays"" realizados",int
8,fans,Numero de fans del usuario,int
9,elite,"Años en los que el usuario fue ""elite"" en Yelp",object


In [9]:
reviews_dict

,Variable,Descripcion,Tipo
0,review_id,Identificador de la critica o reseña,string
1,user_id,Identificador del usuario,string
2,business_id,Identificador del negocio,string
3,stars,Valoración del negocio,float
4,date,Dia en el que se realizo la critica o reseña. ...,string
5,text,Contenido de la critica o reseña,string
6,useful,Numero de votos utiles recibidos,int
7,funny,Numero de votos divertidos recibidos,int
8,cool,"Numero de votos ""guays"" recibidos",int


## Data loading

In [10]:
# Paths to files:
data_files = ['../data/yelp_academic_dataset_business.json',
         '../data/yelp_academic_dataset_checkin.json',
         '../data/yelp_academic_dataset_tip.json',
         '../data/yelp_academic_dataset_user.json',
         '../data/yelp_academic_dataset_review.json']

# Data types:
business_dtypes = {"business_id": str, 
            "name": str, 
            "adress": str,
            "city": str, 
            "state": str, 
            "adress": str,
            "postal code": str, 
            "latitude": np.float32, 
            "longitude": np.float32,
            "stars": str, 
            "review_count": np.int32, 
            "is_open": str,
            "attributes": object,
            "categories": object,
            "hours": object} 

checkin_dtypes = {"business_id": str, 
            "date": str}

tip_dtypes = {"text": str, 
            "date": str,
            "compliment_count": np.int32, 
            "business_id": str,
            "user_id": str}

user_dtypes = {"user_id": str,
            "name": str,
            "review_count": np.int32, 
            "yelping_since": str,
            "friends": object,
            "useful": np.int32, 
            "funny": np.int32,
            "cool": np.int32,
            "fans": np.int32, 
            "elite": object,
            "average_stars": np.float32, 
            "compliment_hot": np.int32,
            "compliment_more": np.int32,
            "compliment_profile": np.int32,
            "compliment_cute": np.int32,
            "compliment_list": np.int32,
            "compliment_note": np.int32,
            "compliment_plain": np.int32,
            "compliment_cool": np.int32,
            "compliment_funny": np.int32,
            "compliment_writer": np.int32,
            "compliment_photos": np.int32} 

reviews_dtypes = {"review_id": str, 
            "user_id": str,
            "business_id": str, 
            "stars": str, 
            "date": str,
            "text": str,
            "useful": np.int32, 
            "funny": np.int32,
            "cool": np.int32}

data_types = [business_dtypes, checkin_dtypes, tip_dtypes, user_dtypes, reviews_dtypes]

In [11]:
#Data loading:
business, checkin, tip, users, reviews = load_data(data_files, data_types)

Here we have the 5 datasets:

In [12]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426678,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551125,-90.335693,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880455,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155563,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338184,-75.471657,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [13]:
checkin.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [14]:
tip.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [15]:
users.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [16]:
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


### Tables of interest

We decide to use the business, reviews and users tables since we consider they have more relevant and useful information

#### Business filter

The business dataset is huge, since it contains a lot of different business we are going to check which of them are more frequent in order to choose one

In [17]:
words_counter(business, 'categories', 300)

,word,count
0,&,108359
1,Services,56156
2,Restaurants,52343
3,Food,43813
4,Shopping,24858
...,...,...
295,Pediatric,528
296,Toy,518
297,Notaries,516
298,Gastropubs,513


**We select the 'Restaurants' business**

In [18]:
# Set the list of keywords
keywords = ['Restaurants']

# Dropping business with no category:
restaurants = business.dropna(subset=['categories'])

# Keep only the rows that contain one of the keywords (the business we want to keep)
restaurants = (restaurants[restaurants['categories'].str.lower()
              .str.contains('|'.join([x.lower() for x in keywords]))]
              .reset_index(drop=True))

In [19]:
restaurants.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155563,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269592,-87.058945,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565166,-90.321083,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208103,-86.768173,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456322,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."


As we've dropped a lot of business, we must drop all the id's of those ones for the rest of the tables

### Reviews filter

In [20]:
# Use the isin() method to create a boolean mask
mask = reviews['business_id'].isin(restaurants.business_id)

# Use the mask to index the dataframe and keep only the relevant rows
reviews_filtered = reviews[mask]

### Users filter

In [21]:
# Use the isin() method to create a boolean mask
mask = users['user_id'].isin(reviews_filtered.user_id.tolist())

# Use the mask to index the dataframe and keep only the relevant rows
users_filtered = users[mask]

### Parquet

We are going to save the data extrated in a parquet format

In [22]:
# List of dataframes to convert
dataframes = [restaurants, reviews_filtered, users_filtered]

# List of file names
filenames = ["../data/restaurants.parquet", 
             "../data/reviews.parquet",
             "../data/users.parquet"]

# Convert each dataframe to a Parquet file
save_dfs_to_parquet(dataframes, filenames)